In [2]:
import pandas as pd

##### 0. base 작업
Respiratory 진단 받은 환자의 SUBJECT_ID, HADM_ID 추출

In [3]:
# respiratory 환자 사망률
print("호흡기 질환 환자의 사망률")
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/D_ICD_DIAGNOSES.csv")
word = ["respiratory", "Respiratory"]
disease = '|'.join(word)
df1 = df[df['LONG_TITLE'].str.contains(disease)]
disease_list = df1["ICD9_CODE"].to_list()
print("Respiratory 관련 ICD9 code 수: ", len(disease_list))

호흡기 질환 환자의 사망률
Respiratory 관련 ICD9 code 수:  92


In [4]:
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/DIAGNOSES_ICD.csv")

def add(group):
    return ' '.join(group['ICD9_CODE'].fillna(""))

df1 = df.groupby(['SUBJECT_ID', 'HADM_ID']).apply(add).reset_index(name="ICD9_CODE")

mask = df1['ICD9_CODE'].str.contains('|'.join(disease_list), na=False)
df2 = df1[mask]

# 동일한 환자가 여러 번 입원한 경우 최초 입원 기록만 남기기
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/ADMISSIONS.csv")
df3 = df[["HADM_ID", "ADMITTIME", "DISCHTIME", "DEATHTIME"]]
df4 = pd.merge(df2, df3, on="HADM_ID")
df4['ADMITTIME'] = pd.to_datetime(df4['ADMITTIME'])
df4['DISCHTIME'] = pd.to_datetime(df4['DISCHTIME'])
df4['DEATHTIME'] = pd.to_datetime(df4['DEATHTIME'])
df5 = df4.sort_values(by=['SUBJECT_ID', 'ADMITTIME'], ascending=[True, True])
df5 = df5.drop_duplicates(subset=['SUBJECT_ID'], keep='first')
df5.sort_values(by=["SUBJECT_ID"], ascending=True, inplace=True)
print("Respiratory 환자 수(최초 입원): ", len(df5))

# LOS 추가 후 24H 이상 머문 환자 list만 저장
df5["LOS"] = df5["DISCHTIME"] - df5["ADMITTIME"]
over = df5["LOS"] >= "1days"
df6 = df5[over].sort_values(by=["LOS"], ascending=True)
subject_list = df6["SUBJECT_ID"].to_list()
print("LOS가 24H 이상인 환자 수: ", len(subject_list))

# 첫 입원 시점 연령을 구하기 위해 PATIENTS Table에서 DOB 칼럼 가져오기
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/PATIENTS.csv")
df = df[["SUBJECT_ID", "DOB"]]
df['DOB'] = pd.to_datetime(df['DOB']).dt.date
df7 = pd.merge(df6, df, on="SUBJECT_ID")

# 첫 입원 시점 연령 칼럼 생성
df7["AGE"] = ((df7["ADMITTIME"].dt.date - df7["DOB"])//365).dt.days

# 18세 이상 환자 선별
over = df7["AGE"] >= 18
df8 = df7[over].sort_values(by=["AGE"], ascending=True)
subject_list = df8["SUBJECT_ID"].to_list()

# 89세 이상 환자 제거
over = df8["AGE"] < 89
df9 = df8[over].sort_values(by=["AGE"], ascending=True)
final_list = df9["SUBJECT_ID"].to_list()
print("18세 이상 89세 미만 환자 수: ", len(final_list))

# DOA(Death or Alive): 사망했다면 사망까지 걸린 시간, 생존했다면 NaT 출력
for i in range(len(df9)):
    if df9["DEATHTIME"].iloc[i] != "NaT":
        df9["DOA"] = (df9["DEATHTIME"]-df9["ADMITTIME"])
    else:
        df9["DOA"] = "NaT"

df10 = df9[["SUBJECT_ID", "HADM_ID", "LOS", "AGE", "DOA"]]
df10

Respiratory 환자 수(최초 입원):  10329
LOS가 24H 이상인 환자 수:  9932
18세 이상 89세 미만 환자 수:  7631


,SUBJECT_ID,HADM_ID,LOS,AGE,DOA
6392,20936,157548,16 days 06:15:00,18,NaT
582,25600,122994,2 days 12:49:00,18,NaT
1012,79166,177249,3 days 11:02:00,18,NaT
3214,75775,101668,7 days 13:07:00,18,NaT
5057,26861,115381,11 days 16:01:00,18,NaT
...,...,...,...,...,...
6091,48806,116558,14 days 23:36:00,88,NaT
3008,19067,192309,7 days 01:22:00,88,NaT
2138,40370,199999,5 days 12:33:00,88,NaT
6524,47335,161669,16 days 20:28:00,88,16 days 20:28:00


##### 1. 인구통계학 데이터 추출
1. ADMISSIONS
2. PATIENTS
3. ICU_STAYS
4. TRANSFERS
5. PROCEDURES_ICD
6. DIAGNOSES_ICD

In [5]:
df1 = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/ADMISSIONS.csv")
df2 = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/PATIENTS.csv")

In [6]:
# Respiratory 진단 받은 환자의 AMISSIONS, PATIENTS 합치기
# print(len(df10))
df = pd.merge(df10, df1, on=['SUBJECT_ID', 'HADM_ID'])
# print(len(df))
df = pd.merge(df, df2, on=['SUBJECT_ID'])
# print(len(df))
df = df.drop(["ROW_ID_x", "ROW_ID_y", "ADMITTIME", "DISCHTIME", "DEATHTIME", "DOB", "DOD", "DOD_HOSP", "DOD_SSN", "EXPIRE_FLAG"], axis=1)

print("활력징후 데이터 column: ", len(df.columns))
print("활력징후 데이터 column name: ", df.columns.values)
df

활력징후 데이터 column:  19
활력징후 데이터 column name:  ['SUBJECT_ID' 'HADM_ID' 'LOS' 'AGE' 'DOA' 'ADMISSION_TYPE'
 'ADMISSION_LOCATION' 'DISCHARGE_LOCATION' 'INSURANCE' 'LANGUAGE'
 'RELIGION' 'MARITAL_STATUS' 'ETHNICITY' 'EDREGTIME' 'EDOUTTIME'
 'DIAGNOSIS' 'HOSPITAL_EXPIRE_FLAG' 'HAS_CHARTEVENTS_DATA' 'GENDER']


,SUBJECT_ID,HADM_ID,LOS,AGE,DOA,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,GENDER
0,20936,157548,16 days 06:15:00,18,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Private,NaN,NOT SPECIFIED,SINGLE,WHITE,2151-07-24 02:24:00,2151-07-24 11:13:00,SEPSIS,0,1,M
1,25600,122994,2 days 12:49:00,18,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,SINGLE,WHITE,2183-03-05 02:17:00,2183-03-05 06:43:00,OVERDOSE,0,1,M
2,79166,177249,3 days 11:02:00,18,NaT,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,2154-03-02 00:34:00,2154-03-02 14:36:00,ASTHMA;COPD EXACERBATION,0,1,M
3,75775,101668,7 days 13:07:00,18,NaT,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Private,ENGL,PROTESTANT QUAKER,SINGLE,WHITE,2177-01-30 14:11:00,2177-01-30 18:00:00,PARAPHARYNGEAL ABSCESS,0,1,M
4,26861,115381,11 days 16:01:00,18,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,ENGL,CATHOLIC,SINGLE,WHITE,2109-08-08 19:23:00,2109-08-09 00:58:00,FEVER,0,1,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7626,48806,116558,14 days 23:36:00,88,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,ENGL,JEWISH,MARRIED,WHITE,2135-12-01 18:01:00,2135-12-01 20:12:00,CONGESTIVE HEART FAILURE;PNEUMONIA,0,1,M
7627,19067,192309,7 days 01:22:00,88,NaT,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,Medicare,ENGL,JEWISH,SINGLE,WHITE,2168-08-12 10:30:00,2168-08-12 15:30:00,LUDWIGS ANGINA,0,1,F
7628,40370,199999,5 days 12:33:00,88,NaT,EMERGENCY,CLINIC REFERRAL/PREMATURE,LONG TERM CARE HOSPITAL,Medicare,ENGL,JEWISH,MARRIED,WHITE,2136-04-04 20:29:00,2136-04-05 00:33:00,PNEUMONIA,0,1,M
7629,47335,161669,16 days 20:28:00,88,16 days 20:28:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,ENGL,UNOBTAINABLE,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE,1,1,M


##### 3. Postgre SQL 사용하기

In [ ]:
import psycopg2
import pandas as pd

# 데이터베이스 연결
conn = psycopg2.connect(database='mimic',
                        user='myusername',
                        password='mypassword',
                        host='localhost',
                        port='5432')

# 체중과 키 정보를 가져올 쿼리 작성
query = """
SELECT p.subject_id, p.gender, p.dob, p.dod, p.dod_hosp, p.expire_flag,
    c.hadm_id, c.charttime, c.itemid, c.value, c.valueuom
FROM patients p
INNER JOIN chartevents c
    ON p.subject_id = c.subject_id
WHERE c.itemid IN (763, 3723, 226707) -- 체중, 키 ITEMID
"""

# 쿼리 실행하여 결과를 데이터프레임으로 저장
df = pd.read_sql_query(query, conn)

# 체중, 키 데이터 추출
weight = df[df.itemid.isin([763, 3723])].copy()
weight.loc[weight.itemid == 3723, 'value'] *= 0.453592  # lb -> kg 변환
height = df[df.itemid == 226707].copy()

# 데이터 확인
print(weight.head())
print(height.head())

# 데이터베이스 연결 종료
conn.close()
